In [1]:
import lxml, urllib.request        # lxml is a library for parsing, urllib is to open the page
from bs4 import BeautifulSoup as bs       # bs4 helps decode and search through page.

from requests import get

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [2]:
lists=[]
mainUrl = 'https://www.thehindu.com/news/national/?page='                 # Hindu Website url           

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Referer': 'https://cssspritegenerator.com',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8',
         'Connection': 'keep-alive'}
stories=[]
storylist=0
for x in range(1,6):                                                      # Iterate through first 5 pages
    newsPaper={}
    url = mainUrl + str(x)
    d = get(url, headers=hdr)
    soup = bs(d.content, 'lxml')

    # Parse page and create list of the links to each news 
    links = []
    for news in soup.find_all('div', class_='story-card-news'):

        link = news.find_all('a')
        links.append(link[2]['href'])
    data=[]                                                     # paragraphs of the new and return a story list.
    for i in range(len(links)):
        url = links[i]
        d = get(url, headers=hdr)
        soup = bs(d.content, 'lxml')

        aa = soup.find('div', class_='article')
        bb = aa.find_all('p')
        cc = bb[1].text + bb[2].text
        data.append(cc)
    # Show each story with title as the indices of each correspond 
    for i in data:
        if i not in stories:
            stories.append(i)                                        # correctly, i.e. index zero of both are for same News.



In [3]:
for i in range(len(stories)):
    print(str(i) + '\n' + stories[i]+ ' \n')
    print('________________________________________\n')

0
 A day after unprecedented violence in Delhi during the tractor parade, the government on Wednesday said it has never stated that the doors for dialogue with farmers are closed and stressed that it will inform whenever the decision is taken for fresh talks.“We have never said that the doors for dialogue are closed. Have you heard. Whenever talks are held, we will let you know,” Union Minister Prakash Javadekar said, when asked at the Cabinet briefing on whether the doors for talks with farmers are closed now. 

________________________________________

1
The Cabinet Committee on Economic Affairs on Wednesday approved an increase in the Minimum Support Price (MSP) for copra, Information and Broadcasting Minister Prakash Javadekar said.The decision, taken at a meeting chaired by Prime Minister Narendra Modi, would benefit copra farmers in 12 coastal States, he stated. 

________________________________________

2
The Congress demanded the immediate sacking of Union Home Minister Amit S

In [4]:
from textblob import TextBlob                                                   # the module that runs sentiment analysis                                                                 

In [5]:
def find_Sentiment(val):                                                                                                               
    if val==0:
        return 'Neutral'
    elif val>0:
        return 'Positive'
    else:
        return 'Negative'

In [6]:
sent = []                                                                   # analysis on it and returns 
sentVal = []
subj=[]
c=0
for news in stories:
    analysis = TextBlob(news)
    sentiment = find_Sentiment(analysis.sentiment.polarity)
    scoreval = analysis.sentiment.polarity
    subval=analysis.sentiment.subjectivity
    sentVal.append(scoreval)
    subj.append(subval)
    sent.append(sentiment)
    print(news + "\n" + str(sent[c])+ " "+str(sentVal[c])+ "\n"+str(subval)+"\n")
    c+=1
    print()


     

 A day after unprecedented violence in Delhi during the tractor parade, the government on Wednesday said it has never stated that the doors for dialogue with farmers are closed and stressed that it will inform whenever the decision is taken for fresh talks.“We have never said that the doors for dialogue are closed. Have you heard. Whenever talks are held, we will let you know,” Union Minister Prakash Javadekar said, when asked at the Cabinet briefing on whether the doors for talks with farmers are closed now.
Positive 0.05833333333333333
0.32222222222222224


The Cabinet Committee on Economic Affairs on Wednesday approved an increase in the Minimum Support Price (MSP) for copra, Information and Broadcasting Minister Prakash Javadekar said.The decision, taken at a meeting chaired by Prime Minister Narendra Modi, would benefit copra farmers in 12 coastal States, he stated.
Positive 0.2
0.2


The Congress demanded the immediate sacking of Union Home Minister Amit Shah and alleged that the

In [7]:
print(sent)
print(len(sent))

['Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Negative', 'Positive', 'Negative', 'Neutral', 'Negative', 'Negative', 'Positive', 'Negative', 'Positive', 'Negative', 'Neutral', 'Negative', 'Negative', 'Neutral', 'Positive', 'Neutral', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative']
31


In [8]:
sentiment_score=[sent.count("Positive"),sent.count("Neutral"),sent.count("Negative")]
print(sentiment_score)

[14, 6, 11]


In [9]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
langs = ["Positive","Neutral","Negative"]
ax.bar(langs,sentiment_score)
plt.show()

<Figure size 640x480 with 1 Axes>

In [10]:
import json
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('0w53ZeLXNDXySWlPrj4-gAKbJ5i0k53O-CdS4Jxun5Sf')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)
doctone=[]
sent_tones=[]
tone_analyzer.set_service_url('https://api.eu-gb.tone-analyzer.watson.cloud.ibm.com/instances/7929578f-79e4-4749-8b55-b8dced09fb4f')
for news in stories:
    senttemp=[]
    tone_analysis = tone_analyzer.tone(
        {'text': news},
        content_type='application/json'
    ).get_result()
    d=json.dumps(tone_analysis, indent=2)
    data=json.loads(d)  
    if data['document_tone']['tones']!=[]:
        doctone.append(str(data['document_tone']['tones'][0]['tone_name']))
    else:
        doctone.append("Not found")

    if 'sentences_tone' in data: 
        for sentence in data['sentences_tone']:
            if sentence['tones'] == []:
                pass
            else:
                for m in sentence['tones']:
                    senttemp.append(m['tone_name'])
    sent_tones.append(senttemp)                    
print(sent_tones)
print()
print(doctone)


[['Analytical', 'Confident'], [], ['Sadness', 'Sadness'], ['Anger', 'Joy'], [], [], [], [], ['Analytical', 'Analytical', 'Analytical'], [], [], [], ['Anger', 'Joy', 'Analytical'], ['Joy'], [], ['Joy'], ['Anger', 'Analytical', 'Analytical'], [], [], [], [], [], [], [], [], ['Sadness', 'Analytical', 'Analytical'], ['Analytical', 'Analytical', 'Analytical', 'Analytical', 'Analytical', 'Tentative'], ['Analytical'], ['Joy', 'Joy'], [], ['Sadness']]

['Analytical', 'Not found', 'Sadness', 'Tentative', 'Sadness', 'Sadness', 'Not found', 'Not found', 'Analytical', 'Joy', 'Fear', 'Not found', 'Not found', 'Not found', 'Joy', 'Confident', 'Analytical', 'Sadness', 'Not found', 'Not found', 'Not found', 'Not found', 'Not found', 'Joy', 'Sadness', 'Analytical', 'Analytical', 'Analytical', 'Joy', 'Not found', 'Not found']


In [11]:
emotion=["Anger", "Disgust", "Fear", "Joy","Sadness"]
lang_styles=["Analytical", "Confident","Tentative"]

In [12]:
import nltk
from collections import Counter 
import speech_recognition as sr
import pyttsx3
r = sr.Recognizer()
def speak(command): 
    engine = pyttsx3.init() 
    engine.say(command) 
    engine.runAndWait() 
    
def sent_analysis(text):
    count=0
    docnumber=[]
    for news in stories:
        tokens = nltk.word_tokenize(news.lower())
        if text in tokens:
            docnumber.append(count)
        count+=1
    if len(docnumber)==0:
        speak("Term not found in any news")
        return 
    else:
        for i in docnumber:
            print(i)
            print(sent[i])
            print(doctone[i])
            print(stories[i])
            print("Overall Nature of Document Having "+text+ " is "+ sent[i] + " having the tone as " + doctone[i])
            print()
            speak(stories[i])
            speak("Overall Nature of Document Having "+text+ " is "+ sent[i] + " having the tone as " + doctone[i])
            

def common_words():
    stopword = set(stopwords.words('english'))
    newsString=""
    fil_words=[]

    for news in stories:
        newsString+=news.lower()

    split_it = newsString.split()

    for w in split_it:
        if w not in stopword:
            fil_words.append(w)

    counter = Counter(fil_words)

    most_occur = counter.most_common(15)
    
    print(most_occur)

    for i in range(len(most_occur)):
        speak(most_occur[i][0])
        
            
    
try:
    with sr.Microphone() as source2:
        r.adjust_for_ambient_noise(source2, duration=0.2)
        speak("These are the most commonly used words")
        print("These are the most commonly used words\n")
        common_words()
        speak("Please say the term to find the sentiment")
        print("\nPlease say the term to find the sentiment\n")
        audio2 = r.listen(source2)
        MyText = r.recognize_google(audio2)
        MyText = MyText.lower()
        print("Term: "+ MyText)
        sent_analysis(MyText)
        
except sr.RequestError as e:
    print("Could not process: ".format(e))
    
except sr.UnknownValueError:
    print("Unknown error")

These are the most commonly used words

[('said', 20), ('day', 14), ('minister', 14), ('january', 12), ('farmers', 11), ('union', 10), ('delhi', 9), ('republic', 9), ('covid-19', 9), ('ministry', 8), ('one', 8), ('tractor', 7), ('government', 7), ('indian', 7), ('wednesday', 6)]

Please say the term to find the sentiment

Term: government
0
Positive
Analytical
 A day after unprecedented violence in Delhi during the tractor parade, the government on Wednesday said it has never stated that the doors for dialogue with farmers are closed and stressed that it will inform whenever the decision is taken for fresh talks.“We have never said that the doors for dialogue are closed. Have you heard. Whenever talks are held, we will let you know,” Union Minister Prakash Javadekar said, when asked at the Cabinet briefing on whether the doors for talks with farmers are closed now.
Overall Nature of Document Having government is Positive having the tone as Analytical

2
Positive
Sadness
The Congress de